In [9]:
import numpy as np
import pandas as pd
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 33.9137,
	"longitude": -98.4934,
	"start_date": "2018-01-01",
	"end_date": "2023-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "wind_speed_10m", "wind_speed_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm"],
	"temperature_unit": "fahrenheit"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(5).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(10).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(11).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(12).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm = hourly.Variables(13).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 33.91915512084961°N -98.5286865234375°E
Elevation 291.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                     date  temperature_2m  relative_humidity_2m  \
0     2018-01-01 00:00:00       20.452099             37.061073   
1     2018-01-01 01:00:00       19.822100             34.205345   
2     2018-01-01 02:00:00       19.102100             31.932373   
3     2018-01-01 03:00:00       18.652100             31.029348   
4     2018-01-01 04:00:00       18.382099             31.390779   
...                   ...             ...                   ...   
52579 2023-12-31 19:00:00       53.032101             55.762844   
52580 2023-12-31 20:00:00       54.202103             53.995529   
52581 2023-12-31 21:00:00       54.382103             52.331104   
52582 2023-12-31 22:00:00       53.122101             51.771263   
52583 2023-12-31 23:00:00       50.692101             55.621300   

       apparent_temperature  precipitation  rain  snowfall  snow_dept

In [11]:
hourly_dataframe.to_csv('/Users/amrithkumaar/Downloads/WichitaFalls_weather(NORTH).csv')

In [10]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,wind_speed_10m,wind_speed_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm
0,2018-01-01 00:00:00,20.452099,37.061073,8.841932,0.0,0.0,0.0,0.0,18.430452,28.901073,38.160000,27.922100,37.012100,49.702103,64.462097
1,2018-01-01 01:00:00,19.822100,34.205345,7.660351,0.0,0.0,0.0,0.0,20.192118,31.435646,34.919998,27.922100,36.562099,49.702103,64.462097
2,2018-01-01 02:00:00,19.102100,31.932373,6.883301,0.0,0.0,0.0,0.0,20.124611,31.869658,34.200001,27.922100,36.202099,49.612099,64.462097
3,2018-01-01 03:00:00,18.652100,31.029348,6.639393,0.0,0.0,0.0,0.0,19.211996,30.312773,32.399998,27.922100,35.842102,49.522099,64.462097
4,2018-01-01 04:00:00,18.382099,31.390779,6.963984,0.0,0.0,0.0,0.0,16.946787,27.345785,31.319998,27.832100,35.572102,49.522099,64.462097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,2023-12-31 19:00:00,53.032101,55.762844,45.271301,0.0,0.0,0.0,0.0,18.668005,23.557316,33.119999,51.862099,47.632103,55.102100,69.322098
52580,2023-12-31 20:00:00,54.202103,53.995529,46.514191,0.0,0.0,0.0,0.0,18.584509,23.686249,33.480000,53.572098,48.082100,55.012100,69.232101
52581,2023-12-31 21:00:00,54.382103,52.331104,46.048775,0.0,0.0,0.0,0.0,20.592503,26.427530,35.279999,54.562099,48.532101,55.012100,69.232101
52582,2023-12-31 22:00:00,53.122101,51.771263,44.187630,0.0,0.0,0.0,0.0,21.889065,29.304090,36.360001,54.202103,49.072098,55.012100,69.232101
